In [1]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


### Load Vocab

In [2]:
vocab = {}
idToWord = {}
with open('vocab.txt') as f:
    words = f.read().splitlines()
    for wordIndex in words:
        word, index = wordIndex.split(' -----> ')
        vocab[word] = index
        idToWord[int(index)] = word

### Create input squences

In [3]:
look_back_len = 1000 + 1
sequences = []
vocabulary_size = len(vocab)

with open('int-seq.txt') as f:
    files = f.read().splitlines()
    for file in files:
        numbers = list(map(int, file.split(',')[:-1]))
        #print(numbers)
        for i in range(look_back_len, len(numbers)):
            seq = numbers[i-look_back_len:i]
            sequences.append(seq)
    #print(sequences)

    n_sequences = np.empty([len(sequences), look_back_len], dtype='int32')
    for i in range(len(sequences)):
        n_sequences[i] = sequences[i]

n_sequences

array([[1000,    0,   20, ..., 1000,    3,    0],
       [   0,   20,  614, ...,    3,    0, 1000],
       [  20,  614,    0, ...,    0, 1000,    3],
       ...,
       [   0,    1, 1000, ...,    6,    0,    8],
       [   1, 1000,    3, ...,    0,    8,   16],
       [1000,    3, 1000, ...,    8,   16,    9]])

### Convert output to one hot encoded vector

In [4]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
print(len(train_targets))

train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_targets[0])

40216
[1. 0. 0. ... 0. 0. 0.]


### Train Model

In [5]:
import keras
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.save("model_{}.hd5".format(epoch))

In [6]:
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

saver = CustomSaver()

model.fit(train_inputs,train_targets, epochs = 20 ,verbose=1, validation_split=0.3, callbacks=[saver])
model.save("mymodel.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 1000)        1002000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000, 50)          210200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 1002)              51102     
Total params: 1,286,052
Trainable params: 1,286,052
Non-trainable params: 0
_________________________________________________________________
None


c:\Miniconda\envs\azureml_py36_tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 28151 samples, validate on 12065 samples
Epoch 1/20
28151/28151 [==============================] - 1765s 63ms/step - loss: 3.4138 - accuracy: 0.2464 - val_loss: 2.6892 - val_accuracy: 0.4104
Epoch 2/20
28151/28151 [==============================] - 1759s 62ms/step - loss: 2.6564 - accuracy: 0.4079 - val_loss: 2.2689 - val_accuracy: 0.5213
Epoch 3/20
28151/28151 [==============================] - 1755s 62ms/step - loss: 2.2418 - accuracy: 0.5054 - val_loss: 2.0726 - val_accuracy: 0.5752
Epoch 4/20
28151/28151 [==============================] - 1757s 62ms/step - loss: 2.0091 - accuracy: 0.5490 - val_loss: 2.0149 - val_accuracy: 0.5838
Epoch 5/20
28151/28151 [==============================] - 1764s 63ms/step - loss: 1.8751 - accuracy: 0.5711 - val_loss: 2.0201 - val_accuracy: 0.5754
Epoch 6/20
28151/28151 [==============================] - 1754s 62ms/step - loss: 1.7746 - accuracy: 0.5912 - val_loss: 2.0026 - val_accuracy: 0.5949
Epoch 7/20
28151/28151 [==========================

### Test Model

In [7]:
from keras.preprocessing.sequence import pad_sequences

# sample python input
"""
num = 46
print
"""
encoded_text = [2,13,35,5]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre', value = int(vocab['<pad_token>']))
print(encoded_text, pad_encoded)

print("Top 3 Suggestions:")
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    print(idToWord[i])

[2, 13, 35, 5] [[1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001
  1001 1001 1001 1001 1001 1001 1001 1001 1001 10